# Distributed data parallel RNN-T training with PyTorch and SageMaker distributed

[Amazon SageMaker's distributed library](https://docs.aws.amazon.com/sagemaker/latest/dg/distributed-training.html) can be used to train deep learning models faster and cheaper. The [data parallel](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html) feature in this library (`smdistributed.dataparallel`) is a distributed data parallel training framework for PyTorch, TensorFlow, and MXNet.

This notebook demonstrates how to use `smdistributed.dataparallel` with PyTorch(version 1.10.2) on [Amazon SageMaker](https://aws.amazon.com/sagemaker/) to train an RNN-T model on [`LibriSpeech`](http://www.openslr.org/12) (License: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)) using [Amazon FSx for Lustre file-system](https://aws.amazon.com/fsx/lustre/) as data source.

The outline of steps is as follows:

1. Stage the `LibriSpeech` dataset on [Amazon S3](https://aws.amazon.com/s3/)
2. Create Amazon FSx Lustre file-system and import data into the file-system from S3
3. Build Docker training image and push it to [Amazon ECR](https://aws.amazon.com/ecr/)
4. Configure data input channels for SageMaker
5. Configure hyper-prarameters
6. Define training metrics
7. Define training job, set distribution strategy to `SMDataParallel` and start training

**NOTE:** With large training dataset, we recommend using [Amazon FSx](https://aws.amazon.com/fsx/) as the input file system for the SageMaker training job. FSx file input to SageMaker significantly cuts down training start up time on SageMaker because it avoids downloading the training data each time you start the training job (as done with S3 input for SageMaker training job) and provides good data read throughput.


**NOTE:** This example requires `SageMaker Python SDK v2.X`.

## Amazon SageMaker Initialization

Initialize the notebook instance. Get the AWS Region and a SageMaker execution role.

### SageMaker role

The following code cell defines `role` which is the IAM role ARN used to create and run SageMaker training and hosting jobs. This is the same IAM role used to create this SageMaker Notebook instance. 

`role` must have permission to create a SageMaker training job and host a model. For granular policies you can use to grant these permissions, see [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html). If you do not require fine-tuned permissions for this demo, you can use the IAM managed policy `AmazonSageMakerFullAccess` to complete this demo. 

As described above, since we will be using FSx, please make sure to attach `FSx Access` permission to this IAM role.

In [ ]:
%%time
! python3 -m pip install --upgrade sagemaker
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
role_name = role.split(["/"][-1])
print(f"SageMaker Execution Role:{role}")
print(f"The name of the Execution role: {role_name[-1]}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account:{account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region:{region}")

To verify that the role above has required permissions:

1. Go to the IAM console: https://console.aws.amazon.com/iam/home.
2. Select **Roles**.
3. Enter the role name in the search box to search for that role. 
4. Select the role.
5. Use the **Permissions** tab to verify this role has required permissions attached.

## Prepare SageMaker Training Images

1. SageMaker by default uses the latest [Amazon Deep Learning Container Images (DLC)](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) PyTorch training image. In this step, we use it as a base image and install additional dependencies required for training the RNN-T model.
2. In the GitHub repository https://github.com/HerringForks/SMDDP-Examples/tree/main/pytorch/rnnt we have forked an RNN-T example from [ml commons/training_results_v1.0](https://github.com/mlcommons/training_results_v1.0/tree/master/NVIDIA/benchmarks/rnnt/implementations/pytorch) and adapted the training script to work with `smdistributed.dataparallel`. We will use the `Dockerfile` provided there.

In [ ]:
# login to ecr
! aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com

In [ ]:
# clone the SMDDP-example repo
! rm -rf SMDDP-Examples && git clone https://github.com/HerringForks/SMDDP-Examples && cd SMDDP-Examples/pytorch/rnnt

In [ ]:
# build the image
! cd SMDDP-Examples/pytorch/rnnt && bash scripts/docker/build.sh {region}

In [ ]:
# name the image
image = "rnnt-smdataparallel-sagemaker"  # Example: rnnt-smdataparallel-sagemaker
tag = "pt1.10.2"  # Example: pt1.10.2

In [ ]:
# tag the image we just built and push it to ecr
! chmod +x tag_and_push.sh; bash tag_and_push.sh {region} {image} {tag}

## Preparing FSx Input for SageMaker

1. Download and prepare your training dataset on S3, please refer to the [instructions](https://github.com/mlcommons/training_results_v1.0/tree/master/NVIDIA/benchmarks/rnnt/implementations/pytorch#steps-to-download-data). 
2. Follow the [steps](https://docs.aws.amazon.com/fsx/latest/LustreGuide/create-fs-linked-data-repo.html) to create a FSx linked with your S3 bucket with training data. Make sure to add an endpoint to your VPC allowing S3 access.
3. Follow the [steps](https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/) to configure your SageMaker training job to use FSx.

### Important Caveats

1. You need to use the same `subnet` and `vpc` and `security group` used with FSx when launching the SageMaker notebook instance. The same configurations will be used by your SageMaker training job.
2. Make sure you set [appropriate inbound/output rules](https://docs.aws.amazon.com/fsx/latest/LustreGuide/limit-access-security-groups.html) in the `security group`. Specifically, opening up these ports is necessary for SageMaker to access the FSx file system in the training job. 
3. Make sure `SageMaker IAM Role` used to launch this SageMaker training job has access to `AmazonFSx`.

## SageMaker PyTorch Estimator function options

In the following code block, you can update the estimator function to use a different instance type, instance count, and distribution strategy. You're also passing in the training script you reviewed in the previous cell.

**Instance types**

`SMDataParallel` supports model training on SageMaker with the following instance types only. For best performance, it is recommended you use an instance type that supports Amazon Elastic Fabric Adapter (ml.p3dn.24xlarge and ml.p4d.24xlarge).

1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of `SMDataParallel`, you should use at least 2 instances, but you can also use 1 for testing this example.

**Distribution strategy**

Note that to use DDP mode, you need to update the `distribution` strategy, and set it to use `smdistributed dataparallel`. 

In [ ]:
import os
from sagemaker.pytorch import PyTorch

In [ ]:
instance_type = "ml.p4d.24xlarge"  # "ml.p3dn.24xlarge"  # Other supported instance type: ml.p3.16xlarge, ml.p4d.24xlarge
instance_count = 2  # You can use 2, 4, 8 etc.
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}"  # YOUR_ECR_IMAGE_BUILT_WITH_ABOVE_DOCKER_FILE
username = "AWS"
subnets = ["<subnet_id>"]  # Should be same as Subnet used for FSx. Example: subnet-0f9XXXX
security_group_ids = [
    "<security_group_id>"
]  # Should be same as Security group used for FSx. sg-03ZZZZZZ
job_name = "pytorch-smdataparallel-rnnt"  # This job name is used as prefix to the sagemaker training job. Makes it easy for your look for your training job in SageMaker Training job console.
file_system_id = "<fsx_id>"  # FSx file system ID with your training dataset. Example: 'fs-0bYYYYYY'

In [ ]:
# Configure the RNN-T hyper-parameters
hyperparameters = {
    "batch_size": 48,
    "beta1": 0.9,
    "beta2": 0.999,
    "max_duration": 16.7,
    "val_batch_size": 44,
    "target": 0.058,
    "lr": 0.007,
    "min_lr": 1e-5,
    "lr_exp_gamma": 0.939,
    "epochs": 80,
    "warmup_epochs": 6,
    "hold_epochs": 33,
    "epochs_this_job": 0,
    "ema": 0.995,
    "model_config": "/workspace/rnnt/configs/baseline_v3-1023sp.yaml",
    "seed": 28400,
    "cudnn_benchmark": "",
    "dali_device": "gpu",
    "weight_decay": 1e-3,
    "log_frequency": 1,  #!!
    "val_frequency": 1,
    "grad_accumulation_steps": 1,
    "prediction_frequency": 100000000,
    "weights_init_scale": 0.5,
    "save_at_the_end": "",
    "max_symbol_per_sample": 300,
    "apex_transducer_loss": "fp16",
    "fuse_relu_dropout": "",
    "multi_tensor_ema": "",
    "batch_eval_mode": "cg_unroll_pipeline",
    "apex_transducer_joint": "pack",
    "buffer_pre_alloc": "",
    "ema_update_type": "fp16",
    "amp_level": 2,
    "data_cpu_threads": 8,
    "batch_split_factor": 1,
    "min_seq_split_len": 20,
    "vectorized_sa": "",
    "multilayer_lstm": "",
    "enable_prefetch": "",
    "tokenized_transcript": "",
    "vectorized_sampler": "",
    "dist_sampler": "",
    "apex_mlp": "",
    "pre_sort_for_seq_split": "",
    "jit_tensor_formation": "",
}

In [ ]:
estimator = PyTorch(
    entry_point="entry_point.py",
    role=role,
    image_uri=docker_image,
    source_dir=".",
    instance_count=instance_count,
    instance_type=instance_type,
    py_version="py38",
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters,
    subnets=subnets,
    security_group_ids=security_group_ids,
    debugger_hook_config=False,
    # Training using SageMaker distributed dataparallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
)

In [ ]:
# Configure FSx Input for your SageMaker Training job

from sagemaker.inputs import FileSystemInput

file_system_directory_path = "/<mount_name>/<path_to_dataset>"  # NOTE: '/fsx/' will be the root mount path. Example: '/fsx/rnnt/PyTorch'
file_system_access_mode = "ro"
file_system_type = "FSxLustre"
train_fs = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path,
    file_system_access_mode=file_system_access_mode,
)
data_channels = {"train": train_fs}

In [ ]:
# Submit SageMaker training job
estimator.fit(inputs=data_channels, job_name=job_name)